In [1]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2025-01-22 19:29:20--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.33MB/s    in 0.2s    

2025-01-22 19:29:20 (4.33 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [2]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [1]:
!pip install flwr['simulation']

In [3]:
# @title Bibliotecas

import pandas as pd
import numpy as np
import flwr as fl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# @title Leitura dos dados
links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

### Visualização prévia dos dados

In [9]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [10]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [28]:
data.head()

,movieId,imdbId,tmdbId,title,genres,userId,rating,timestamp
0,1,114709,862.0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1,4,0.192846
1,1,114709,862.0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",5,4,0.027210
2,1,114709,862.0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",7,5,0.392449
3,1,114709,862.0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",15,3,0.961643
4,1,114709,862.0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",17,5,0.672945


### Manipulação dos dados


In [5]:
data = links.merge(movies, on="movieId").merge(ratings, on="movieId")
data['genres'] = data['genres'].str.split('|')
data = data.drop(columns=['imdbId', 'tmdbId', 'title'])

generos_explodidos = data.explode('genres')

# Criar colunas binárias para cada gênero
generos_binarios = pd.get_dummies(generos_explodidos['genres'], dtype=int)

# Reagrupar os dados ao nível do filme (somando as colunas binárias)
generos_binarios_agrupados = generos_binarios.groupby(generos_explodidos.index).max()

# Concatenar as colunas binárias de volta ao DataFrame original
df = pd.concat([data.drop(columns='genres'), generos_binarios_agrupados], axis=1)
df = df.drop(columns='(no genres listed)')

In [54]:
print(df)

        movieId  userId  rating   timestamp  Action  Adventure  Animation  \
0             1       1     4.0   964982703       0          1          1   
1             1       5     4.0   847434962       0          1          1   
2             1       7     4.5  1106635946       0          1          1   
3             1      15     2.5  1510577970       0          1          1   
4             1      17     4.5  1305696483       0          1          1   
...         ...     ...     ...         ...     ...        ...        ...   
100831   193581     184     4.0  1537109082       1          0          1   
100832   193583     184     3.5  1537109545       0          0          1   
100833   193585     184     3.5  1537109805       0          0          0   
100834   193587     184     3.5  1537110021       1          0          1   
100835   193609     331     4.0  1537157606       0          0          0   

        Children  Comedy  Crime  ...  Film-Noir  Horror  IMAX  Musical  \
0

In [56]:
# @title Treino rápido para testar (não federado)

# Separar features e labels
X = df.drop(columns=['rating'])
y = df['rating']

# Train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalonar dados numéricos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Definição da rede
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1) # Problema de regressão
])

# Compilar modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Treinar
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1) # Adjust epochs and batch size as needed

# Avaliar
loss, mae = model.evaluate(X_test, y_test)
print(f"Erro Absoluto Médio: {mae}\nErro Quadrado Médio: {loss}\n")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2269/2269 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.5496 - mae: 0.9495 - mse: 1.5496 - val_loss: 1.0389 - val_mae: 0.8081 - val_mse: 1.0389
Epoch 2/10
2269/2269 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.0146 - mae: 0.7961 - mse: 1.0146 - val_loss: 1.0421 - val_mae: 0.8008 - val_mse: 1.0421
Epoch 3/10
2269/2269 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.0065 - mae: 0.7935 - mse: 1.0065 - val_loss: 1.0430 - val_mae: 0.8016 - val_mse: 1.0430
Epoch 4/10
2269/2269 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 1.0075 - mae: 0.7936 - mse: 1.0075 - val_loss: 1.0250 - val_mae: 0.8032 - val_mse: 1.0250
Epoch 5/10
2269/2269 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.0019 - mae: 0.7906 - mse: 1.0019 - val_loss: 1.0348 - val_mae: 0.7952 - val_mse: 1.0348
Epoch 6/10
2269/2269 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.9998 - mae: 0.7920 - mse: 0.9998 - val_loss: 1.0387 - val_mae: 0.7949 - val_mse: 1.0387
Epoch 7/10
2269/2269 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.9989 - mae: 0.7912 - ms

### Federado

In [11]:
# Média ponderada
def weighted_average(metrics):
    maes = [num_examples * m["mae"] for num_examples, m in metrics]
    mses = [num_examples * m["mse"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"mae": sum(maes) / sum(examples),
             "mses": sum(mses) / sum(examples)}

# Classe do Cliente Flower
class MovieLensClient(fl.client.NumPyClient):
    def __init__(self, x_train, y_train, x_val, y_val):
        self.model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
            layers.Dense(32, activation='relu'),
            layers.Dense(1)
        ])
        self.model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=1, verbose=0)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, mae = self.model.evaluate(self.x_val, self.y_val, verbose=0)
        return loss, len(self.x_val), {"mae": mae, "mse": loss}


# Adequação dos dados
X = df.drop(columns=['rating'])
y = df['rating']

# Train/Test + Escalonamento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Splits de dados ficticios (os nossos serão não-iid)
client1_data = (X_train[:len(X_train)//2], y_train[:len(y_train)//2])
client2_data = (X_train[len(X_train)//2:], y_train[len(y_train)//2:])

# Gerador de clientes
def client_fn(cid):
  if cid == "0":
    return MovieLensClient(client1_data[0], client1_data[1], X_test[:len(X_test)//2], y_test[:len(y_test)//2])
  elif cid == "1":
    return MovieLensClient(client2_data[0], client2_data[1], X_test[len(X_test)//2:], y_test[len(y_test)//2:])
  else:
    return None

# Start Flower simulation
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1,  # 100% dos clientes para treino
    fraction_evaluate=1, # 100% dos clientes para teste
    min_fit_clients=2,
    min_evaluate_clients=2,
    min_available_clients=2,
    evaluate_metrics_aggregation_fn=weighted_average
)

# Dados do treinamento sao salvos em 'history'
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=2,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy
)

print(history.metrics_distributed)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2025-01-22 20:42:04,721	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0, 'object_store_memory': 3948082790.0, 'memory': 7896165582.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_c

{'mae': [(1, 0.8047880232334137), (2, 0.8020028471946716), (3, 0.7998130321502686)], 'mses': [(1, 1.0536407828330994), (2, 1.027620017528534), (3, 1.027431845664978)]}


In [12]:
history

History (loss, distributed):
	round 1: 1.0536407828330994
	round 2: 1.027620017528534
	round 3: 1.027431845664978
History (metrics, distributed, evaluate):
{'mae': [(1, 0.8047880232334137),
         (2, 0.8020028471946716),
         (3, 0.7998130321502686)],
 'mses': [(1, 1.0536407828330994),
          (2, 1.027620017528534),
          (3, 1.027431845664978)]}